In [92]:
from sqlalchemy import create_engine, text
from urllib import parse
import pyodbc
import csv
import pandas as pd
from datetime import date
from datetime import timedelta
import common_funcs
import os
from pretty_html_table import build_table

In [58]:
# SQLAlchemy connection
server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = os.getenv('SQL_UID')
pwd = parse.quote_plus(os.getenv('SQL_PWD'))
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)

# Database connection settings
CONNECTION_STRING = (
    f"Driver={{SQL Server}};"
    f"Server=tn-sql;"
    f"Database=autodata;"
    f"UID={os.getenv('SQL_UID')};"
    f"PWD={os.getenv('SQL_PWD')};"
)

In [98]:
print('Connecting to FileMaker Server, it will take a bit\n\n')
cnxn = pyodbc.connect('DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC')
cursor = cnxn.cursor()


Connecting to FileMaker Server, it will take a bit




In [86]:
sql = """
            SELECT Ourpart,"Band A Part Number", "Housing A Part Number",
                "Screw Part Number", "Band Feed from Band data",
                "Ship Diam Max QA Alternate", "Ship Diam Min", "Hex Size", "Band_Thickness", "Band_Width",
                "CameraInspectionRequired", "ScrDrvChk", "Cutout1"
            FROM tbl8Tridon
            WHERE  ("Band Feed from Band data" IS NOT NULL)
                AND (Ourpart IS NOT NULL) AND (RIGHT(Ourpart,1) <> '\r')
                AND ("Housing A Part Number" IS NOT NULL)
                AND (RIGHT("Housing A Part Number",1) <> '\r')
                AND ("Screw Part Number" IS NOT NULL)
                AND (RIGHT("Screw Part Number",1) <> '\r')
                AND ("Band A Part Number" IS NOT NULL)
                AND (RIGHT("Band A Part Number",1) <> '\r')
                AND ("Band_Thickness" IS NOT NULL)
                AND ("Band Width" IS NOT NULL)
                AND (CameraInspectionRequired IS NOT NULL)
                AND (ScrDrvChk IS NOT NULL)
                AND ("Hex Size" IS NOT NULL)
            ORDER BY Ourpart
        """



In [ ]:
cursor.execute(sql)
result = cursor.fetchall()

In [99]:
df = pd.read_sql(sql, cnxn)
BASE_PATH = "\\Inetpub\\ftproot\\acmparts_alt\\parts.csv"
df[["Band Feed from Band data", "Band_Width"]] = df[["Band Feed from Band data", "Band_Width"]].round(2)
df.to_csv(BASE_PATH, index=False, quoting=csv.QUOTE_NONE, header=False)




C:\PycharmProjects\IdealProd\venv310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [66]:
def write_dataframe_to_sql(df, table_name):
    """Write a DataFrame to a SQL Server table."""
    try:
        print("Connecting to SQL Server...")

        # Write DataFrame to SQL Server
        df.to_sql(name=table_name, con=engine, schema='production', if_exists='replace', index=False)
        print(f"Data successfully written to {table_name}.")
    except Exception as e:
        print(f"Error writing DataFrame to SQL Server: {e}")
    # finally:
    #     engine.dispose()


In [67]:
write_dataframe_to_sql(df, "parts_cutout")

Connecting to SQL Server...
Data successfully written to parts_cutout.


In [35]:
test = df.groupby("CameraInspectionRequired")
print(test.size())

CameraInspectionRequired
NO     8412
YES     817
Yes     180
dtype: int64
